# Graphs in PySpark


this is a work in progress by Yaniv. 2025-02-03

Let's dive into Spark's tool for large scale graph processing - ***GraphX library***. We will ... ***(TODO)***


NOAM: you need to relate to the notebook in the sdg folder ( I simply downloaded code from the book)

## Graphs - Revisiting

**(TODO: Revision on Graphs)**

## Graphs in PySpark

Before we use GraphFrames, we need to have the right java package (JAR) installed.
Luckily, it is already installed for us 


In [ ]:
!pip install graphframes # Install the python glue library to the java lib

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Graph Processing") \
    .master("local[*]")\
    .config("spark.jars","/home/jars/graphframes-0.8.4-spark3.5-s_2.12.jar") \
    .getOrCreate()

sc = spark.sparkContext


In [ ]:
from graphframes import GraphFrame

# Create vertices and edges
vertices = spark.createDataFrame([
    ("1", "Alice"),
    ("2", "Bob"),
    ("3", "Charlie")
], ["id", "name"])

edges = spark.createDataFrame([
    ("1", "2", "friend"),
    ("2", "3", "follow"),
    ("3", "1", "friend")
], ["src", "dst", "relationship"])

# Create GraphFrame
g = GraphFrame(vertices, edges)

# Show vertices and edges
print("Vertices:")
g.vertices.show()

print("Edges:")
g.edges.show()
